In [9]:
import socket
import sqlite3
import threading
import unittest
import os
import pickle
import json
import io
import time
import pandas as pd

In [10]:
class QueryBuilder:
    def __init__(self):
        self.command_map = {
            "select": self.select,
            "insert": self.insert,
            "delete": self.delete,
            "update": self.update,
            "create_table": self.create_table,
            "connect": self.connect,
            "search": self.search
        }

    def query(self, table, command_type, *args):
        command_type = command_type.lower()
        try:
            return self.command_map[command_type](table, *args)
        except KeyError:
            raise sqlite3.Error(f"The command: {command_type} was not found.")
        except Exception as e:
            raise sqlite3.Error(f"An error occurred: {e}")

    def select(self, table, fields='*', condition=''):
        self.validate_table_and_fields(table, fields)
        query = f"SELECT {fields} FROM {table}"
        query += f" WHERE {condition}" if condition else ""
        return query

    def insert(self, table, fields, values):
        self.validate_table_fields_values(table, fields, values)
        pickled_values = list(values)
        if len(values) >= 3:
            pickled_values[2] = pickle.dumps(values[2]) 
        if len(values) >= 4:
            pickled_values[3] = pickle.dumps(values[3]) 

        placeholders = ', '.join(['?'] * len(pickled_values))
        query = f"INSERT INTO {table} {fields} VALUES ({placeholders})"
        return query, tuple(pickled_values)

    def delete(self, table, condition):
        self.validate_table_and_condition(table, condition)
        query = f"DELETE FROM {table} WHERE {condition}"
        return query

    def update(self, table, set_data, condition):
        self.validate_table_set_data_condition(table, set_data, condition)
        set_clause = ', '.join([f"{key} = ?" for key in set_data.keys()])
        query = f"UPDATE {table} SET {set_clause} WHERE {condition}"
        return query, tuple(set_data.values())

    def create_table(self, table, fields):
        self.validate_table_and_fields(table, fields)
        query = f"CREATE TABLE {table} ({fields})"
        return query

    def connect(self, _, database_name):
        assert database_name, "Database name cannot be empty."
        return f"CONNECT TO DATABASE {database_name}"

    def search(self, table, search_term):
        self.validate_table_and_fields(table, search_term)
        query = f"SELECT * FROM {table} WHERE name LIKE ?"
        return query, (f'%{search_term}%',)

    def validate_table_and_fields(self, table, fields):
        assert table and fields, "Table name and fields cannot be empty."

    def validate_table_fields_values(self, table, fields, values):
        assert table and fields and values, "Table name, fields, and values cannot be empty."

    def validate_table_and_condition(self, table, condition):
        assert table and condition, "Table name and condition cannot be empty."

    def validate_table_set_data_condition(self, table, set_data, condition):
        assert table and set_data and condition, "Table name, set data, and condition cannot be empty."

In [11]:
class SqliteDB:
    def __init__(self, db_name, json_file=None, table_name=None):
        self.db_name = db_name  # Use the db_name directly without reassigning
        self.json_file = json_file
        self.table_name = table_name
        
        if self.json_file:
            self.load_data()
            self.create_table()
            self.insert_dataframe()  # Call the new method to insert data

    def load_data(self):
        with open(self.json_file) as f:
            raw_data = json.load(f)
            self.df = pd.DataFrame(raw_data)
            self.df.rename(columns={"Name": "Dwarf"}, inplace=True)

    def create_table(self):
        with sqlite3.connect(self.db_name) as conn:
            self.conn = conn
            self.df.to_sql(self.table_name, conn, if_exists='replace', index_label='id')

    def insert_dataframe(self):
        """Inserts all rows of the DataFrame into the database."""
        with sqlite3.connect(self.db_name) as conn:
            self.df.to_sql(self.table_name, conn, if_exists='append', index=False)
            # Or use explicit row insertion, depending on the desired behavior

    def insert_row_column(self, row_data):
        with self.conn:
            self.conn.execute(
                f"INSERT INTO {self.table_name} (Dwarf, Distance, Period) VALUES (?, ?, ?)",
                (row_data['Dwarf'], row_data['Distance'], row_data['Period'])
            )

    def KVfunction(self, action_name):
        return self.actions.get(action_name, lambda: "Unknown action.")()

    def select_all(self):
        return self.query_builder.select(self.table_name).execute(self.conn)

    def select_condition(self, condition):
        return self.query_builder.select(self.table_name).where(condition).execute(self.conn)


In [12]:
class BitstringConverter:
    def __init__(self, input_string):
        if not isinstance(input_string, str):
            raise ValueError("Input must be a string.")
        self._input_string = input_string
        self._conversions = {
            "Bits": self._to_bits,
            "Bytes": self._to_bytes,
            "BitString": self._to_bitstring,
            "ByteArray": self._to_bytearray,
            "BytesIO": self._to_bytesio,
            "From Bits": self._from_bits,
            "From Bytes": self._from_bytes,
        }

    def _to_bits(self):
        return ' '.join(f'{ord(c):08b}' for c in self._input_string)

    def _to_bytes(self):
        return ' '.join(f'{b:02x}' for b in self._input_string.encode('utf-8'))

    def _to_bitstring(self):
        return f"0b{' '.join(format(ord(c), '08b') for c in self._input_string)}"

    def _to_bytearray(self):
        return ' '.join(f'{b:02x}' for b in bytearray(self._input_string, 'utf-8'))

    def _to_bytesio(self):
        bytes_io = io.BytesIO(self._input_string.encode('utf-8'))
        return ' '.join(f'{b:02x}' for b in bytes_io.getvalue())

    def _from_bits(self, bits):
        try:
            byte_array = bytearray(int(bits[i:i + 8], 2) for i in range(0, len(bits), 9))  
            return byte_array.decode('utf-8')
        except ValueError:
            return "Invalid Bit Stream"

    def _from_bytes(self, byte_data):
        try:
            return bytes.fromhex(byte_data.replace(" ", "")).decode('utf-8')
        except (ValueError, AttributeError):
            return "Invalid Byte Stream"

    def convert(self, conversion_type):
        func = self._conversions.get(conversion_type)
        if func:
            if conversion_type in ["From Bits", "From Bytes"]:
                return func(self._input_string)
            return func()
        return "Unknown Conversion Type"

    def __str__(self):
        return f"BitstringConverter(input_string='{self._input_string}')"

    def __repr__(self):
        return f"BitstringConverter('{self._input_string}')"

    def __iter__(self):
        return iter(self._input_string)

    def __len__(self):
        return len(self._input_string)

    def __getitem__(self, index):
        return self._input_string[index]

    def __contains__(self, item):
        return item in self._input_string

    def __eq__(self, other):
        if isinstance(other, BitstringConverter):
            return self._input_string == other._input_string
        return False

In [13]:
class SocketServer:
    def __init__(self, host='localhost', port=12345, db_name='Database.db'):
        self.host = host
        self.port = port
        self.db_name = db_name
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.server_socket.bind((self.host, self.port))
        self.server_socket.listen(5)
        print(f"Server listening on {self.host}:{self.port}")

    def handle_client(self, conn, addr):
        print(f"Connected by {addr}")
        with sqlite3.connect(self.db_name) as db_conn:
            cursor = db_conn.cursor()
            while True:
                data = conn.recv(1024)
                if not data:
                    break
                query = pickle.loads(data)
                print(f"Received query: {query}")
                try:
                    if query[1] is None:
                        cursor.execute(query[0])  # Execute without parameters if None
                    else:
                        cursor.execute(query[0], query[1])  # Execute with parameters if provided

                    db_conn.commit()
                    result = cursor.fetchall() if cursor.description else []
                    response = pickle.dumps(result) + b'\x06'  # ACK at the end
                except sqlite3.Error as e:
                    response = str(e).encode('utf-8') + b'\x00'  # NAK at the end
                conn.send(response)
        print(f"Connection with {addr} closed.")
        conn.close()

    def start(self):
        while True:
            conn, addr = self.server_socket.accept()
            client_thread = threading.Thread(target=self.handle_client, args=(conn, addr))
            client_thread.start()

In [14]:
class SocketClient:
    def __init__(self, host='localhost', port=12345):
        self.host = host
        self.port = port
        self.client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.client_socket.connect((self.host, self.port))
        print(f"Connected to server at {self.host}:{self.port}")

    def send_query(self, query, values=None):
        serialized_query = pickle.dumps((query, values))
        self.client_socket.sendall(serialized_query)
        response = self.client_socket.recv(4096)

        # Decode and handle based on response type
        if response.endswith(b'\x06'):
            try:
                return pickle.loads(response[:-1])
            except (pickle.UnpicklingError, EOFError):
                return response[:-1].decode('utf-8')  # If not pickle, decode as text
        elif response.endswith(b'\x00'):
            return response[:-1].decode('utf-8')  # NAK indicates error in UTF-8 text
        return response


    def close(self):
        self.client_socket.close()

In [24]:
class TestSocketServerClient(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        cls.server = SocketServer(port=12344, db_name='TestDatabase.db')
        cls.db = SqliteDB(db_name='TestDatabase.db', json_file='test_data.json', table_name='TestTable')
        cls.client = SocketClient('localhost', 12344)
        
        # Start server in a separate thread
        cls.server_thread = threading.Thread(target=cls.server.start, daemon=True)
        cls.server_thread.start()
        time.sleep(1)  # Allow server to initialize

    def test_binary_file_transfer(self):
        binary_data = b'\x00\x01\x02\x03Sample binary data\x04\x05\x06'
        response = self.client.send_query(('INSERT INTO TestTable (column1) VALUES (?)', (binary_data,)))
        self.assertIn(binary_data, response.encode() if isinstance(response, str) else response)

    def test_error_handling(self):
        response = self.client.send_query(('INVALID SQL QUERY', None))
        self.assertTrue(response.endswith('\x00'))

    def test_interactive_text_mode(self):
        response = self.client.send_query(('SELECT * FROM TestTable', None))
        self.assertIsInstance(response, list)

    @classmethod
    def tearDownClass(cls):
        cls.client.close()
        cls.server.server_socket.close()


In [25]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

Server listening on localhost:12344
Connected to server at localhost:12344
Connected by ('127.0.0.1', 63380)


Exception in thread Thread-10:
Traceback (most recent call last):
  File "/Users/sophia/miniforge3/envs/jupyter/lib/python3.9/threading.py", line 954, in _bootstrap_inner
    self.run()
  File "/Users/sophia/miniforge3/envs/jupyter/lib/python3.9/threading.py", line 892, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-13-fb1731046366>", line 23, in handle_client
TypeError: argument 1 must be str, not tuple


Received query: (('INSERT INTO TestTable (column1) VALUES (?)', (b'\x00\x01\x02\x03Sample binary data\x04\x05\x06',)), None)


KeyboardInterrupt: 